<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/(%EA%B8%B0%ED%83%80)%EB%85%BC%EB%AC%B8%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
from nltk import word_tokenize, sent_tokenize
import re

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%cd /content/gdrive/My\ Drive/

/content/gdrive/My Drive


In [7]:
data = pd.read_excel('./마이크로바이옴_논문_최종데이터_수정_20230222.xlsx')

In [8]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Title', 'trans_title', 'Author(s)',
       'Date', 'Keywords Plus', 'trans_keywords', 'Publication Year',
       'Full Author Name(s)', 'Organization', 'Abstract', 'trans_abstract',
       'Linked Author Names and Addresses', 'ISSN', 'Document Type',
       'Times Cited', 'Standardized Subject Category', 'Country',
       'Funding Acknowledgement Details', 'Funding Text',
       'Organization Address', 'Subject Category', 'trans_Subject Category',
       '과제명_new', '연구내용_new', 'keyword_sum', '질병', '신체부위', '대상', '1.기반',
       '2. 탐색연구(인과, 상호작용, 연관성, 변화 등)', '3. 기전규명', '4. 후보물질발굴/치료법',
       '5. 비임상/전임상', '6.임상', '7.바이오마커', '식품 등 소재(제거예정)', '노이즈(노이즈=0, 정상=1)',
       '기타', 'cleaned_title', 'cleaned_abs', 'disease', 'body_parts', '미생물',
       'Organ_Country'],
      dtype='object')

In [11]:
data['Abstract'].isnull().sum()

8312

In [12]:
print(data.shape)
data = data.dropna(subset='Abstract')
print(data.shape)

(69070, 46)
(60758, 46)


In [14]:
# 노이즈가 아닌 것만 선택
data = data[data['노이즈(노이즈=0, 정상=1)'] == 1]
print(data.shape)

(39847, 46)


In [15]:
data = data.reset_index()

### 2. 훈련데이터 전처리

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
# 각 문장에 대해 구두점을 제거하고 대문자를 소문자로 변환
normalized_text = []
for string in data['Abstract'].tolist():
  tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
  normalized_text.append(tokens)

In [45]:
data['cleaned'] = normalized_text

In [47]:
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [49]:
tagged_corpus_list = []
for index, row in tqdm(data.iterrows(), total=len(data)):
  text = row['Abstract']
  tag = row['Title']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=word_tokenize(text)))

100%|██████████| 39847/39847 [01:28<00:00, 451.28it/s]


In [50]:
len(tagged_corpus_list)

39847

In [51]:
tagged_corpus_list[0]

TaggedDocument(words=['Purpose', 'of', 'Review', 'Resident', 'microbial', 'communities', 'likely', 'modify', 'risk', 'for', 'allergic', 'disorders', ',', 'including', 'food', 'allergy', '.', 'We', 'review', 'epidemiologic', 'studies', 'linking', 'microbial', 'exposures', 'to', 'food', 'allergy', 'risk', 'and', 'discuss', 'the', 'mechanisms', 'by', 'which', 'the', 'microbiome', 'may', 'modulate', 'oral', 'tolerance', '.', 'We', 'additionally', 'address', 'ongoing', 'translational', 'efforts', 'in', 'human', 'microbiome', 'studies', '.', 'Recent', 'Findings', 'Epidemiologic', 'studies', 'and', 'murine', 'models', 'support', 'that', 'altered', 'microbial', 'exposures', 'and', 'colonization', 'in', 'early', 'life', 'modify', 'food', 'allergy', 'risk', '.', 'Differential', 'microbiota', 'confer', 'protection', 'or', 'susceptibility', 'to', 'food', 'allergy', 'by', 'modulating', 'the', 'regulatory', 'tone', 'of', 'the', 'mucosal', 'immune', 'system', '.', 'Recent', 'efforts', 'are', 'focused

### 3. Doc2Vec 학습 및 테스트

In [52]:
from gensim.models import doc2vec

In [53]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag size: {len(model.docvecs.doctags.keys())}", end=' / ')

Tag size: 39802 / 

In [55]:
# 모델 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)
model.save('microbiome.doc2vec')

In [57]:
data['Title'][0]

'Role of the Microbiome in Food Allergy'

In [58]:
data['Abstract'][0]

'Purpose of Review Resident microbial communities likely modify risk for allergic disorders, including food allergy. We review epidemiologic studies linking microbial exposures to food allergy risk and discuss the mechanisms by which the microbiome may modulate oral tolerance. We additionally address ongoing translational efforts in human microbiome studies. Recent Findings Epidemiologic studies and murine models support that altered microbial exposures and colonization in early life modify food allergy risk. Differential microbiota confer protection or susceptibility to food allergy by modulating the regulatory tone of the mucosal immune system. Recent efforts are focused on the identification of bacterial strains necessary for oral tolerance in human and microbial-based clinical trials. Summary Early childhood appears to be critical for the colonization of a diverse microbiota necessary for the induction and maintenance of oral tolerance. Identification and functional evaluation of p

In [59]:
# 테스트
model.docvecs.most_similar(data['Title'][0], topn=20)

[('FECAL MICROBIOTA TRANSPLANTATION (FMT) FOR IRRITABLE BOWEL SYNDROME (IBS): A SYSTEMATIC REVIEW AND META-ANALYSIS',
  0.42192596197128296),
 ('PREDICTORS OF TREATMENT FAILURE IN FECAL MICROBIOTA TRANSPLANT FOR CLOSTRIDIUM DIFFICILE INFECTION',
  0.4217663109302521),
 ('ORAL AND FECAL MICROBIOME CHARACTERISTICS IN TREATMENT NAIVE PEDIATRIC IBD PATIENTS OVER TIME: A PROSPECTIVE LONGITUDINAL STUDY',
  0.4188489019870758),
 ('ROLE OF TOLL-LIKE RECEPTOR 5 IN THE BRAIN-GUT-MICROBIOTA TRIALOGUE: A NEW PHARMACOLOGICAL TARGET FOR THE TREATMENT OF CHRONIC ABDOMINAL PAIN AND ASSOCIATED COMORBIDITIES',
  0.4182617962360382),
 ('ALTERNATION OF GUT MICROBIOTA IN PATIENT WITH IMMUNE CHECKPOINT INHIBITORS',
  0.4169010818004608),
 ('COLONIC MUCOSAL MICROBIOME IS ASSOCIATED WITH MUCOSAL MICRORNA EXPRESSION IN IRRITABLE BOWEL SYNDROME',
  0.4166880249977112),
 ('IMPACT OF EPITHELIAL BARRIER DISRUPTION ON MUCUS LAYER AND GUT MICROBIOTA IN MICE',
  0.4152335524559021),
 ('DIFFERENCES IN FECAL MICROBIOME

### 4. 클러스터링

In [72]:
from sklearn.cluster import KMeans

In [73]:
cluster_dict = {i:[] for i in range(0, 8)}
cluster_dict_title = {i:[] for i in range(0, 8)}

In [74]:
clustering_method = KMeans(n_clusters=8, n_init=1, init='k-means++', random_state=0)

In [75]:
X = model.docvecs.vectors_docs
clustering_method.fit(X)

KMeans(n_init=1, random_state=0)

In [76]:
clustering_method.labels_

array([7, 7, 7, ..., 7, 7, 7], dtype=int32)

In [78]:
for text_tag, label in zip(tagged_corpus_list, clustering_method.labels_):
  text, tag = text_tag
  cluster_dict[label].append(text)
  cluster_dict_title[label].append(tag)

In [79]:
for i in range(8):
  print(f"label {i} size: {len(cluster_dict_title[i])}")

label 0 size: 1
label 1 size: 1
label 2 size: 1
label 3 size: 1
label 4 size: 1
label 5 size: 1
label 6 size: 1
label 7 size: 39795


제대로 분류가 안됨

In [81]:
cluster_dict_title[0]

[["The regulation effect of EGCG3''Me phospholipid complex on gut flora of a high-fat diet-induced obesity mouse model"]]

In [82]:
cluster_dict_title[1]

[['The Interaction Between the Microbiome and Tumors']]